In [2]:
import sqlite3
import pandas as pd
from pandasql import sqldf
conn = sqlite3.connect('hackathon.db')
c = conn.cursor()

# Extract all records related to senders including device info and message info
mega1 = pd.read_sql('''
                    SELECT devices.*, senders.sender_id, message_id, send_date, send_time, content, senders.name, senders.username, senders.phone
                    FROM devices 
                    JOIN sms_messages AS sms ON devices.device_id = sms.sender_id 
                    JOIN senders ON sms.sender_id = senders.sender_id
                    ORDER BY sms.sender_id, send_date, send_time''', conn)

# Extract all records related to receivers including device info anf message info
mega15 = pd.read_sql('''
                    SELECT devices.*, receivers.receiver_id, message_id, send_date, send_time, content, receivers.name, receivers.username, receivers.phone
                    FROM devices 
                    JOIN sms_messages AS sms ON devices.device_id = sms.receiver_id 
                    JOIN receivers ON sms.receiver_id = receivers.receiver_id
                    ORDER BY sms.receiver_id, send_date, send_time''', conn)

# Join the tables on message_id adding suffixes to differentiate the duplicate columns
mega1 = mega1.merge(mega15, on='message_id', how='inner', suffixes=('_x', '_y'))

# Create dim_senders where each row represents a single sender. Include all device information 
dim_senders = sqldf('''
                    SELECT DISTINCT sender_id, name_x AS name, username_x AS username, phone_x AS phone, product_name_x AS product_name, serial_number_x AS serial_number, model_number_x AS model_number, software_version_x AS software_version
                    FROM mega1
                         ''')

# preview the staging table after tranformation
mega2 = sqldf('''
                SELECT device_id_x, device_id_y, message_id, sender_id, receiver_id, send_date_y, send_time_y, name_y, username_y, phone_y, product_name_y, serial_number_y, content_x, content_y, model_number_y, software_version_y
                FROM mega1
                ORDER BY sender_id, send_date_y, send_time_y''')

# Create dim_senders where each row represents a single receiver. Include all device information
dim_receivers = sqldf('''
                    SELECT DISTINCT receiver_id, name_y AS name, username_y AS username, phone_y AS phone, product_name_y AS product_name, serial_number_y AS serial_number, model_number_y AS model_number, software_version_y AS software_version
                    FROM mega1
                         ''')

# preview the staging table after transformation
mega3 = sqldf('''
                SELECT device_id_x, device_id_y, message_id, sender_id, receiver_id, send_date_x, send_time_x, content_x, content_y
                FROM mega1
                ORDER BY sender_id, send_date_x, send_time_x''')

# remove all duplicate columns or unneeded columns
final = sqldf('''
              SELECT message_id, sender_id, receiver_id, send_date_x AS send_date, send_time_x AS send_time, content_x AS content
              FROM mega1
              ''')
print('Staging Table Before')
display(mega1.head())
print('Create dim_senders Table')
display(dim_senders.head())
print('Staging Table After')
display(mega2.head())
print('Create dim_receivers Table')
display(dim_receivers.head())
print('Staging Table After')
display(mega3.head())
print('Resulting Fact Table')
display(final.head())

Staging Table Before


,device_id_x,product_name_x,serial_number_x,model_number_x,software_version_x,sender_id,message_id,send_date_x,send_time_x,content_x,...,serial_number_y,model_number_y,software_version_y,receiver_id,send_date_y,send_time_y,content_y,name_y,username_y,phone_y
0,1,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3,1,60026,2020-01-12,07:28:48,Whatcha doin?,...,6DU5XNSKBJ,FUU4B7B,16.1.1,949,2020-01-12,07:28:48,Whatcha doin?,Kermie Raise,kraiseqc,1009751448
1,1,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3,1,46311,2020-01-25,16:45:06,LOL,...,X7JD7AX597,7NDUYHB,12.1.3,49,2020-01-25,16:45:06,LOL,Beauregard Nettleship,,2228428590
2,1,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3,1,16541,2020-02-01,02:42:41,Pick up your mess next time!,...,GFEZ828CVX,DHBUEQW,15.9.0,730,2020-02-01,02:42:41,Pick up your mess next time!,Zia Hucker,zhuckerk9,7445535081
3,1,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3,1,10534,2020-02-03,03:59:56,What do you mean?,...,DCLS3STFYE,H9YAV8R,16.1.1,118,2020-02-03,03:59:56,What do you mean?,Ollie Druett,odruett39,2771816367
4,1,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3,1,11566,2020-02-05,09:48:20,Oof that's gotta hurt,...,S4KFZNR8QQ,4AW/6AX,15.9.4,220,2020-02-05,09:48:20,Oof that's gotta hurt,Dorice Iwanczyk,diwanczyk63,5091426043


Create dim_senders Table


,sender_id,name,username,phone,product_name,serial_number,model_number,software_version
0,1,Gabriella Glynne,,4198827962,Apple iPhone 12,NMQJ4T2NMN,3SLEZXA,12.1.3
1,2,Ber Dallyn,bdallyn1,4859206090,Apple iPhone 14,E237XAQB3X,BTGL84X,16.0.2
2,3,Ariella Christmas,,1602545501,Apple iPhone 12 Plus,AJDANUTU45,CHENTB2,16.1.2
3,4,Myles Kettlestringes,mkettlestringes3,4634207778,Apple iPhone 13 Plus,8BYHQUR3UW,R42RCB4,16.0.0
4,5,Blinni Fullilove,bfullilove4,1238414855,Apple iPhone 13,RRV3658XNE,PN8R28/,16.0.1


Staging Table After


...

,receiver_id,name,username,phone,product_name,serial_number,model_number,software_version
0,949,Kermie Raise,kraiseqc,1009751448,Samsung Galaxy S21,6DU5XNSKBJ,FUU4B7B,16.1.1
1,49,Beauregard Nettleship,,2228428590,Apple iPhone 13,X7JD7AX597,7NDUYHB,12.1.3
2,730,Zia Hucker,zhuckerk9,7445535081,Apple iPhone 13,GFEZ828CVX,DHBUEQW,15.9.0
3,118,Ollie Druett,odruett39,2771816367,Apple iPhone 13,DCLS3STFYE,H9YAV8R,16.1.1
4,220,Dorice Iwanczyk,diwanczyk63,5091426043,Apple iPhone 12 Plus,S4KFZNR8QQ,4AW/6AX,15.9.4


Staging Table After


,device_id_x,device_id_y,message_id,sender_id,receiver_id,send_date_x,send_time_x,content_x,content_y
0,1,949,60026,1,949,2020-01-12,07:28:48,Whatcha doin?,Whatcha doin?
1,1,49,46311,1,49,2020-01-25,16:45:06,LOL,LOL
2,1,730,16541,1,730,2020-02-01,02:42:41,Pick up your mess next time!,Pick up your mess next time!
3,1,118,10534,1,118,2020-02-03,03:59:56,What do you mean?,What do you mean?
4,1,220,11566,1,220,2020-02-05,09:48:20,Oof that's gotta hurt,Oof that's gotta hurt


Resulting Fact Table


,message_id,sender_id,receiver_id,send_date,send_time,content
0,60026,1,949,2020-01-12,07:28:48,Whatcha doin?
1,46311,1,49,2020-01-25,16:45:06,LOL
2,16541,1,730,2020-02-01,02:42:41,Pick up your mess next time!
3,10534,1,118,2020-02-03,03:59:56,What do you mean?
4,11566,1,220,2020-02-05,09:48:20,Oof that's gotta hurt
